##  감성체인 생성

In [8]:
import os
import time
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from concurrent.futures import ThreadPoolExecutor, as_completed
from langchain.chains import LLMChain  # LLMChain을 사용하여 Chain을 구성합니다
import logging

In [9]:
# 로깅 설정
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger()

In [10]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# OpenAI LLM 모델 설정
llm = ChatOpenAI(openai_api_key=api_key, model_name="gpt-4-turbo-preview", temperature=0.3)

# 감성 분석 기본 프롬프트
prompt_template = PromptTemplate.from_template(
    """你是一名研究中国的社会科学家，你的任务是分析中国新闻对“美日韩”三边关系的报道，并从中国的视角分析这些报道的情感**。

    **评分标准**：
    - 强烈负面 (-5): 极端消极
    - 负面 (-3 ~ -4): 较为消极
    - 轻微负面 (-1 ~ -2): 略微消极
    - 中立 (0): 纯粹的事实陈述，不带有任何情感倾向。
    - 轻微正面 (+1 ~ +2): 略微积极
    - 正面 (+3 ~ +4): 较为积极
    - 强烈正面 (+5): 极端积极

    **请严格分析以下新闻文章的所有句子，并为每个句子提供一个情感分数。 句子应根据句号（。）进行分隔，不得遗漏任何句子**。
    **新闻文章内容:**
    {article_content}

    **请返回每个句子的情感分数，每行一个分数，不要包含任何额外的文本、注释或解释。**
    """
)

In [11]:
# 체인 구성
sentiment_chain = LLMChain(prompt=prompt_template, llm=llm)

## 데이터 로드

In [16]:
import json

# JSON 파일 로드
file_path = "../data/0308/샘플.json"
with open(file_path, "r", encoding="utf-8") as f:
    data = json.load(f)

print(f"총 {len(data)}개의 기사 로드 완료")


총 5개의 기사 로드 완료


## 감성분석 수행 함수

In [13]:
# 감성 분석 수행 함수
def analyze_sentiment(article):
    """기사 전체 본문을 입력하고 문장별 감성 점수를 분석하는 함수"""
    time.sleep(1.5)  # Rate Limit 방지를 위해 대기
    try:
        # 기사 본문만 전달
        input_data = {"article_content": article["cleaned_content"]}

        # `sentiment_chain.run()`을 사용하여 데이터 처리
        response = sentiment_chain.invoke(input_data)  # input_data 전달

        print(f"Response format: {response}")

        # 'text' 키에서 감성 분석 결과를 추출하여 'sentiment_responses'에 저장
        if isinstance(response, dict):
            if 'text' in response:
                article["sentiment_responses"] = response['text'].strip()  # 감성 분석 결과 저장
            else:
                article["sentiment_responses"] = "Error: 'text' key not found in response"
        else:
            article["sentiment_responses"] = "Error: Response is not a dictionary"

        return article
    except Exception as e:
        article["sentiment_responses"] = f"Error: {str(e)}"  # 오류 발생 시 오류 메시지 저장
        return article  # 오류 발생 시에도 응답 내용 저장

def process_article(article):
    """GPT 응답 후처리"""
    sentiment_response = article.get("sentiment_responses", "")

    # 문장이 없는 경우 바로 반환
    if not sentiment_response:
        return article

    sentiment_scores = []
    for line in sentiment_response.split("\n"):
        try:
            # 문장별 감성 점수를 처리
            sentiment_scores.append(float(line.strip()))  # 각 점수 추가
        except (ValueError, IndexError):
            continue
    article["sentiment_scores"] = sentiment_scores
    article["avg_sentiment_score"] = round(sum(sentiment_scores) / len(sentiment_scores), 2) if sentiment_scores else None
    return article

def run_sentiment_analysis(data, output_file):
    """JSON 파일을 로드하여 감성 분석을 병렬 실행하고 결과를 저장하는 함수"""
    results = []

    # 기사별 병렬 실행 (최대 3개 스레드 사용)
    with ThreadPoolExecutor(max_workers=3) as executor:
        future_to_article = {executor.submit(analyze_sentiment, article): article for article in data}

        for future in as_completed(future_to_article):
            article = future_to_article[future]  # 원본 기사 가져오기
            processed_article = process_article(article)  # 후처리
            if processed_article is not None:
                results.append(processed_article)  # 결과 저장

            # 중간 저장 (기사별로 즉시 JSON 파일 업데이트)
            with open(output_file, "w", encoding="utf-8") as f:
                json.dump(results, f, ensure_ascii=False, indent=4)
            print(f" '{article['title']}' 분석 완료.")
    print(f" 전체 기사 감성 분석 완료. {output_file}")

In [17]:
output_path = "../data/0308/results_smaple_본문별입력_문장별분석.json"
run_sentiment_analysis(data, output_path)

2025-03-08 14:13:30,757 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'article_content': '刘 琳 日本首相岸田文雄访美期间美国日本菲律宾在华盛顿举行首次三边峰会。分析人士表示这是美日菲举行国家安全顾问会晤海警联合演练之后深化安全合作的又一动作也是美国完善在印太地区小多边机制网络的重要一步。 尽管美方公开表示此次峰会和美日菲合作不针对中国但从会后三国发表的联合声明看其指向性非常明显。声明炒作南海东海等问题对中国维护国家主权的行为表示所谓关切强调加强海上安全领域的合作决定建立三边海事对话新机制计划在南海组织更多联合巡逻。透过该声明美国企图纠合盟友搅动地区安全局势的居心显露无遗。 除安全合作外三国联合声明还提及包容性经济增长基础设施建设产业链供应链高新技术等领域的合作。此举意味着美国不仅要在安全领域还要在经济科技等领域拉拢盟友。 此次峰会的召开美日菲三国各有考量。从美国方面看拜登政府高度重视盟友伙伴在其安全布局中的作用。美在所谓的印太战略下不断推进联盟体系的网络化试图通过拉拢地区国家构建一系列小多边机制遏制打压中国。日本菲律宾是这个同盟网络的重要节点。从日本方面看基于海上通道安全及介入南海问题的考虑日本近年来着力提升与菲律宾越南等国的关系加入美日菲三边机制为其推进东南亚布局提供了又一个重要抓手。对菲律宾来说无论是推进同日本的互惠准入协定谈判还是允许美军深度利用分布在吕宋岛等地区的军事基地都说明菲律宾企图倚靠域外力量获得筹码。 美日菲三边合作的机制化将对地区和平稳定及中国周边海上安全形势带来复杂影响。美国一直谋求以体系力量与中国竞争实现所谓的竞赢。新的美日菲三边机制形成对美国已建构的美日印澳四方安全对话美英澳三边安全伙伴关系美日韩三国同盟等小多边机制构成补充。美不断拱火递刀搅动地区局势希望推动相关国家成为其介入地区局势的代理人国际社会应有清醒的认知并保持高度警惕。 作者单位中共中央党校国家行政学院国际战略研究院', 'text': '-1\n-2\n-3\n-4\n-3\n-2\n-3\n-3\n-4\n-4\n-4\n-5'}
 '美强化“小多边”机制危害地区和平稳定_光明网 - 军事频道' 분석 완료.


2025-03-08 14:13:34,877 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'article_content': '不久前日本首相岸田文雄访美并与拜登会晤双方同意大幅强化军事同盟将地区冲突风险进一步推高 美日加紧勾连险恶用心昭然若揭 美日铁拳2024联合军事演习中一架日本陆上自卫队CH47支奴干直升机降落在美海军两栖攻击舰飞行甲板上。 日本民众在东京举行守护宪法集会反对岸田政府扩军备武。 近日日本首相岸田文雄访问美国并在华盛顿与美国总统拜登举行会晤双方同意深化美日同盟关系加强军事合作还极力渲染阵营对抗拉拢亚太国家搞小圈子。美日加紧勾连企图搅乱亚太进而谋取私利的险恶用心昭然若揭。 全面升级同盟关系 据媒体报道此次岸田访美期间美日对日美安保条约进行了60余年来最大规模的升级双方将通过强化驻日美军与自卫队协同扩展多边合作联合研发防卫装备等方式推动军事同盟关系不断深化。 重组驻日美军指挥机构是此次日美安保条约升级的重点拜登将其称为美日同盟建立后最重要的改进。目前驻日美军的指挥调动由美军印太司令部负责驻日美军司令部的职权仅限于指导美日联合演训协调日美地位协定的运用等。此次调整主要涉及两方面内容一方面美方将赋予驻日美军司令部更多权限另一方面日本拟于2024年末设立统一指挥陆海空自卫队的统合司令部。这两个步骤完成后驻日美军司令部将与统合司令部进行直接协调并共享信息。据媒体报道美军目前正在考虑任命一名四星上将领导驻日美军以便与身份对等的统合司令部负责人对接。 拓展美日多边合作。近年来美国在亚太地区推行所谓印太战略拉拢地区国家搞小圈子而日本正是美国推行该战略的重要盟友。此次升级同盟关系美国意在将美日同盟打造成推进印太战略的枢纽。在美国的撮合下美国日本菲律宾领导人在岸田访美期间举行首次三边峰会。三方会后发布的声明说除帮助菲律宾建设铁路港口等基础设施外美日承诺继续支持菲海岸警卫队能力建设支持菲国防现代化建设。三国还将通过举行海上联合军事演习等方式推进防务合作。 加强防卫装备领域相关合作。岸田此访期间美日双方还就防卫装备合作达成一系列协议包括新设磋商机制探讨防卫装备的共同研发和生产以及允许美国大型舰艇在日本进行维修等。此外美日发表的联合声明中还包括建设美日低轨道卫星网拦截高超音速滑翔飞行器强化美日澳在无人僚机上的技术合作等内容。 相互利用各有算盘 当前国际格局正在经历前所未有的深刻变化。在这样的时代背景下美日领导人就升级同盟一拍

2025-03-08 14:14:10,989 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'article_content': '环球时报环球网报道 记者 邢晓婧 刘欣日本首相岸田文雄当地时间8日抵达美国首都华盛顿开启其任内首次访美行程。这也是继已故日本前首相安倍晋三后日本首相时隔约9年再次以国宾身份访问美国。9日受访专家在接受环球时报记者采访时表示岸田文雄访美日美双方各有盘算美国需要日本充当其在亚洲的马前卒维护既得利益而日本需要确保无论美国大选结果如何都不会影响日美同盟关系。虽然双方标榜将在经济领域展开深入合作但避谈颇具争议的日本收购美铁案仍为岸田文雄的美国之行蒙上阴影。在谈及本周举行的首次美日菲三方领导人会议时相关专家认为这种小团体的存在恰恰是东亚地区和平稳定的破坏者搅局者是和平环境中最大的变数。 日美首脑会谈各有盘算 据日本经济新闻9日报道岸田文雄计划在当地时间9日前往阿灵顿国家公墓献花会见包括微软总裁在内的美国经济界人士10日参加白宫的欢迎仪式与美国总统拜登举行首脑会谈召开共同记者会并出席官方晚宴11日在美国国会发表演讲主题为美日领导力与面向未来的合作同日举行首次美日菲三方领导人会议12日参观丰田汽车公司在美国南部北卡罗来纳州建设的汽车电池厂目的是促进日本公司在美投资14日返回日本。 在岸田文雄的访美行程中日美首脑会谈受到广泛关注。岸田文雄在行前已表示此次会谈对于向世界展示牢固的日美同盟而言意义重大。日本广播协会NHK9日报道称在日益严峻的国际形势下日美首脑会谈将深化双方在防卫领域的合作预计改善各自指挥和控制流程以促进自卫队和美军之间更顺畅的合作。日美两国还将同意建立防卫装备磋商框架促进共同开发并允许美国大型舰艇在日本维修。此外拟在日美首脑会谈的联合声明中将日美关系定位为全球伙伴关系。 日本经济新闻援引外务省官员的话透露称岸田文雄此行意在强化日美安保和经济关系确保在11月美国总统大选前继续执行两国间的主要政策以免大选结果影响日美关系。在中国实力提升的情况下日本要加强在安保经济等领域和美国的同盟关系。日本担心若美国前总统特朗普再次当选或为日美关系带来不确定因素。 中国社会科学院日本研究所副所长吕耀东9日在接受环球时报记者采访时表示美国大选带来的不确定性是日本不愿看到的日方希望此访强化日美同盟的共识并使这种共识具有可持续性即便政党轮替也不会有太大变化。 岸田文雄访美双方各有盘算。吕耀东表示美国在东亚的既得利益需要盟

2025-03-08 14:14:13,957 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'article_content': '新华社北京4月10日电 题伸向亚太的黑手起底美国主导北约加速东进亚太之害 新华社记者刘赞 1949年4月4日在美国主导下北大西洋两岸的12个西方国家签署北大西洋公约北大西洋公约组织成立。75年来这个以防御苏联入侵为由而建立的军事联盟在冷战结束后不断越界扩权突破自身条约规定的地理范围近年来加速东进亚太。 美国是北约亚太化的幕后推手为一己私利将北约作为推行印太战略贩卖安全焦虑的霸权主义战略工具加剧地区紧张局势。北约已经搞乱了欧洲现在开始威胁亚太数十年来繁荣稳定的局面。 加速染指亚太 日本产经新闻日前报道美国政府计划邀请日本首相岸田文雄出席定于今年7月在美国华盛顿举行的北约峰会这将是岸田连续第三年参加北约峰会。 2022年6月日本韩国澳大利亚新西兰四个亚太国家领导人首次出席在西班牙马德里举行的北约峰会。2023年7月在立陶宛维尔纽斯举行的北约峰会上北约成员国领导人同亚太四国领导人会晤向外界展示其关系变得更加紧密。 2006年时任美国驻北约大使维多利亚纽兰首次提出全球伙伴关系概念试图通过与日韩澳新四国建立联系国机制扩大北约的全球影响力。2012年至2014年北约与日韩澳新分别签署个别伙伴关系与合作计划IPCP。2014年北约为让其他国家参与北约领导的军事行动提出伙伴关系互操作性倡议日韩澳新均加入该倡议框架下的互操作性平台。近年来特别是美国拜登政府上台以来北约东进亚太的步伐明显加快。 2021年北约出台2030年议程提出要积极介入全球事务特别是印太事务。2022年4月北约外长会邀请四国外长参加。随后便有了四国领导人同年6月首次参加北约马德里峰会的一幕。2023年7月北约与日韩澳三国将合作关系由IPCP升级为量身定制的合作伙伴项目ITPP合作范围进一步扩大。北约还试图在日本东京设立联络处作为其在亚太地区的首个办事机构只是由于法国反对才不得不搁置。北约一些下属机构也积极吸纳亚太国家。韩国和日本2022年加入北约联合网络防御示范中心。澳大利亚和印度也已加入该机制。 近年来北约国家增加了在亚太地区的军事活动。2021年英国派遣伊丽莎白女王号航母打击群赴亚太其舰队中还包括美国和荷兰舰艇。美国海军学会新闻网站刊文称这赋予了这支舰队真正的北约特征。英国在该航母打击群赴亚太期间还宣布在亚太地区永久部署两艘军舰。同年德

2025-03-08 14:14:20,420 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Response format: {'article_content': '美日韩近日宣布正式启动实时共享导弹预警情报系统这不仅标志着三方强化所谓威慑能力取得重大进展也成为美国整合亚太军事同盟体系的最新成果。在拜登政府提出的一体化威慑战略指引下美国在亚太地区紧锣密鼓推进联盟威慑战略依托盟友强化前沿军力部署增加军备投入升级战备态势显著推升了地区动荡和冲突风险。 2021年4月美国国防部长奥斯汀抛出一体化威慑概念强调将以全军种全领域全政府全手段全盟伴的一体化方式整合动员内外资源和力量以压倒性军力优势达到慑止对手的目的。从推行两年多来的进展看亚太地区成为该战略的主攻方向以联盟为主核常兼备攻防一体为特点的一体化军事威慑体系正在浮现。 在美国日前通过2024财年国防授权法案后美参众两院军事委员会宣称为扩大对华战略竞争中的优势地位必须不断强化印太地区军事同盟关系。从军事布局上看美国依托亚太盟友北中南三锚体系已初步成型。 北锚是美日韩三边军事安全合作。以今年4月美韩华盛顿宣言和8月美日韩戴维营峰会为标志美国强化对日韩的延伸威慑承诺派战略核潜艇和战略轰炸机赴朝鲜半岛活动美日韩重启三边联合军演建成导弹预警情报共享系统恍如冷战期间的美日韩铁三角重现三边军事同盟呼之欲出。中锚是美菲同盟和美日菲三边安全合作。今年以来美菲重新拉紧同盟关系制定首个防务指导方针菲向美军新开4个军事基地美国防部拨款1亿美元建设在菲军事基地。日本则积极推动商签美日菲三边安全协议11月与菲律宾启动准同盟性质的互惠准入协定谈判谋求将菲打造为搅局南海剑指台海的军事据点。南锚是美英澳奥库斯三边同盟。今年三方进一步扩大合作范畴除了美英向澳提供核动力潜艇这一第一支柱外还提出聚焦关键尖端技术合作的第二支柱谋求在人工智能量子技术网络和太空领域拉其他盟友入伙服务遏华战略需要。 在亚太联盟威慑战略布局之外美国还在大力武装关岛军事基地一方面强化其反导能力构建由萨德爱国者宙斯盾标准3和标准6组成的一体化防空反导体系2024年将形成初始作战能力。另一方面强化其攻击战备提升战略枢纽功能。美军在升级扩充岛上机场设施建造新的潜艇维修码头海军陆战队也在修建兵营和作战训练场计划2024年开始接纳来自冲绳的5000名陆战队员。美太平洋陆军发言人不久前声称美军计划2024年在印太地区部署陆基中程导弹包括陆基版标准6导弹和战斧巡航导弹等射程最远可达2700